# 3D Visualization of Binary Masks

This notebook loads a reference segmentation mask and yours, applies required transformations same as what been used for data preprocessing, and creates a 3D visualization of the binary masks to check if the orientation of your segmentation mask/image is correct.

In [14]:
import nibabel as nib
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from scipy import ndimage
from monai.transforms import Compose, LoadImaged, Orientationd, Spacingd, CropForegroundd, SpatialPadd
pio.renderers.default = 'iframe_connected'

transforms = Compose([
    LoadImaged(["REF", "MOV"], image_only=True, ensure_channel_first=True),
    Orientationd(["REF", "MOV"], axcodes="RAS"),
    Spacingd(["REF", "MOV"], [2.0, 2.0, 2.0], mode="nearest"),
    CropForegroundd("REF", source_key="REF"),
    CropForegroundd("MOV", source_key="MOV"),
    SpatialPadd(["REF", "MOV"], (128, 128, 128), method="symmetric", mode="minimum")
])

### Load Masks

In [15]:
# Load NIfTI files
msk_ref = r'template/cap_ref.nii.gz'
msk_mov = '/mnt/data/Experiment/Data/raw_data/datasets_abdul/private_dataset/nifiti_sax/SAX/predictions/1/1_frame0_0000_gt.nii.gz'

# Get data from NIfTI files
data = transforms({"REF": msk_ref, "MOV": msk_mov})
data_ref = data["REF"].get_array()[0]
data_mov = data["MOV"].get_array()[0]

### Optional Code for RV-Myo Addition

Use the code below only if your segmentation only have right ventricle cavity mask but no myocardium mask, and only if the index of your segmentation mask is arranged as: background -- 0, left ventricle cavity -- 1, left ventricle myocardium --2, right ventricle cavity -- 3, etc.

In [16]:
# Convert segmentation to binary mask (values 2 or 4)
hold = np.isin(data_mov, [2])
mask = np.isin(data_mov, [3])

# Inflate mask_B by ~1.5 pixels in all directions
kernel = np.ones((3, 3, 3))
inflated_mask = ndimage.binary_dilation(mask, structure=kernel)

# Ensure the inflated mask doesn't exceed the original volume size
mask = inflated_mask[:mask.shape[0], :mask.shape[1], :mask.shape[2]]

# Add right ventricular myocardium to the segmentation mask
mask = mask ^ np.isin(data_mov, [3])
data_mov[mask] = 4
data_mov[hold] = 2

### Create Plotly Visualisation

In [17]:
# Convert segmentation to binary mask (values 2 or 4)
mask_ref = np.isin(data_ref, [2, 4])
mask_mov = np.isin(data_mov, [2, 4])

# Get coordinates of non-zero voxels
coords_ref = np.array(np.where(mask_ref)).T
coords_mov = np.array(np.where(mask_mov)).T

# Randomly select 5000 points for visualization
np.random.shuffle(coords_ref)
np.random.shuffle(coords_mov)
coords_ref = coords_ref[:5000]
coords_mov = coords_mov[:5000]

In [18]:
# Create 3D scatter plots
trace_ref = go.Scatter3d(
    x=coords_ref[:, 1],
    y=coords_ref[:, 0],
    z=coords_ref[:, 2],
    mode='markers',
    marker=dict(size=2, color='red', opacity=0.8),
    name='Mask REF'
)

trace_mov = go.Scatter3d(
    x=coords_mov[:, 1],
    y=coords_mov[:, 0],
    z=coords_mov[:, 2],
    mode='markers',
    marker=dict(size=2, color='blue', opacity=0.8),
    name='Mask MOV'
)

# Create the 3D plot
fig = go.Figure(data=[trace_ref, trace_mov])

# Update layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    title='3D Visualization of Binary Masks'
)

# Display the figure
fig.show()

# Save the figure as an HTML file
fig.write_html("binary_masks_3d.html")